In [1]:
# %% [code]
# Instalación de dependencias necesarias
!pip install transformers diffusers gTTS moviepy openai-whisper gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:

In [ ]:

# %% [code]
# 1. Generación de guion con un modelo GPT-2 en español
from transformers import AutoModelForCausalLM, AutoTokenizer

modelo_nombre = "datificate/gpt2-small-spanish"  # Modelo gratuito en español
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
model = AutoModelForCausalLM.from_pretrained(modelo_nombre)

def generar_guion(tema, max_length=200):
    prompt = f"Escribe un guion explicativo sobre {tema} en tono educativo."
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, do_sample=True, temperature=0.7)
    guion = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return guion

# %% [code]
# 2. Conversión de guion a voz con gTTS
from gtts import gTTS

def generar_audio(texto, archivo_salida="audio.mp3"):
    tts = gTTS(texto, lang="es")
    tts.save(archivo_salida)
    return archivo_salida

# %% [code]
# 3. Generación de imagen con Stable Diffusion
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.to("cuda")

def generar_imagen(prompt, archivo_salida="imagen.png"):
    imagen = pipe(prompt).images[0]
    imagen.save(archivo_salida)
    return archivo_salida

# %% [code]
# 4. Edición y ensamblaje del video con MoviePy
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips

def crear_video(imagenes, audio, salida="video.mp4", duracion_por_imagen=5):
    clips = [ImageClip(img).set_duration(duracion_por_imagen) for img in imagenes]
    video = concatenate_videoclips(clips, method="compose")
    audio_clip = AudioFileClip(audio)
    video = video.set_audio(audio_clip)
    video.write_videofile(salida, fps=24)
    return salida

# %% [code]
# 5. Generación de subtítulos automáticos con Whisper
import whisper

model_whisper = whisper.load_model("small")  # Modelo pequeño para prototipos

def generar_subtitulos(audio_file, subtitulo_salida="subtitulos.srt"):
    resultado = model_whisper.transcribe(audio_file)
    with open(subtitulo_salida, "w", encoding="utf-8") as f:
        f.write(resultado["text"])
    return subtitulo_salida

# %% [code]
# 6. (Opcional) Narrador Virtual con Wav2Lip
# No implementado


# %% [code]
# 7. Integración final con Gradio
import gradio as gr

def generar_video_explicativo(tema):
    # Paso 1: Generar guion
    guion = generar_guion(tema)

    # Paso 2: Generar audio a partir del guion
    archivo_audio = "audio.mp3"
    generar_audio(guion, archivo_salida=archivo_audio)

    # Paso 3: Generar imagen ilustrativa; acá usamos el tema para formar el prompt
    prompt_imagen = f"Ilustración explicativa sobre {tema}, arte digital"
    archivo_imagen = "imagen.png"
    generar_imagen(prompt_imagen, archivo_salida=archivo_imagen)

    # Paso 4: Crear video combinando la imagen y el audio
    archivo_video = "video.mp4"
    crear_video([archivo_imagen], archivo_audio, salida=archivo_video, duracion_por_imagen=10)

    # Paso 5: Generar subtítulos (archivo SRT)
    archivo_subtitulos = generar_subtitulos(archivo_audio)

    # Devuelve la ruta del video final y el archivo de subtítulos
    return archivo_video, archivo_subtitulos, guion

# Creamos la interfaz con Gradio: entrada: tema (texto); salidas: video, subtítulos y el guion generado.
iface = gr.Interface(
    fn=generar_video_explicativo,
    inputs=gr.Textbox(label="Tema del Video", placeholder="Ej: La historia del internet"),
    outputs=[
        gr.File(label="Video Explicativo"),
        gr.File(label="Subtítulos (.srt)"),
        gr.Textbox(label="Guion Generado")
    ],
    title="Generador Automático de Videos Explicativos",
    description="Ingresa un tema y el sistema genera un guion, audio, imagen y video explicativo utilizando modelos gratuitos."
)

iface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

...

vae%2Fconfig.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  if event.key is 'enter':

100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 103MiB/s]
  checkpoint = torch.load(fp, map_location=device)



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63a491da547747de82.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
